apply 메서드는 사용자가 작성한 함수를 한 번에 데이터 프레임의 각 행과 열에 적용하여 실행할 수 있게 도와주는 메서드  
즉 함수를 데이터프레임에 브로드캐스팅 해야하는 경우에 apply 메서드를 사용  
apply 메서드를 사용하지 않아도 for 문을 이용하면 되지만, apply 메서드가 속도면에서 더 장점이 있기 때문에 알아두어야하는 메서드

# 제곱 함수와 n 제곱 함수 만들기

In [1]:
def my_sq(x):
    return x ** 2

In [2]:
def my_exp(x, n):
    return x ** n

In [3]:
print(my_sq(4))
print(my_exp(2, 4))

16
16


# 시리즈와 apply 메서드

In [4]:
import pandas as pd 

In [5]:
df = pd.DataFrame({'a':[10, 20, 30], 'b':[20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [6]:
# a 열에 제곱하기
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
# my_sq 는 1개의 인자를 받는 함수
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


2개의 인자값을 받는 함수의 경우 apply 함수에 2번째 인자를 같이 넣어주면 됨.  
시리즈의 값들은 함수의 첫번째 인자 값으로 들어가게 됨.  

In [10]:
# 2개의 인자를 받는 my_exp 함수사용 
ex = df['a'].apply(my_exp, n=2)
print(ex)

ex = df['a'].apply(my_exp, n=3)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64
0     1000
1     8000
2    27000
Name: a, dtype: int64


# 데이터 프레임과 apply 메서드

In [11]:
df = pd.DataFrame({'a':[10, 20, 30], 'b':[20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [12]:
# 값을 전달 받아 출력하는 함수
def print_me(x):
    print(x)

In [16]:
# 함수를 열방향으로 적용할지, 행방향으로 적용할지 정해야함. (axis=0, 1)
df.apply(print_me, axis=0)
df.apply(print_me, axis=1)

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64


0    None
1    None
2    None
dtype: object

In [17]:
# 3개의 인자를 작아 평균을 계산하는 함수 
def avg_3(x, y, z):
    return (x + y + z) / 3

avg 함수에서 df['a'], df['b'] 가 1개의 인자로만 전달되었기 때문에 y, z 에 해당하는 인자값이 전달되지 않았다고 오류가 발생하게 됨.  
따라서 열 단위로 데이터를 처리할 수 있도록 수정해야함.  

In [19]:
df.apply(avg_3)

TypeError: avg_3() missing 2 required positional arguments: 'y' and 'z'

In [31]:
# 평균 함수 재정의
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    
    return (x + y + z) / 3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [30]:
# 행의 개수를 보다 일반적으로 처리하는 함수
def avg_3_applt(col):
    sum = 0
    for item in col:
        sum += item
    return sum / df.shape[0]

행 방향으로 처리할려면 shape[0] 을 shape[1] 로 바꾸어 주면 됨.  

In [33]:
# 행 방향으로 처리하는 함수 
def avg_3_axis1(row):
    sum = 0
    for item in row:
        sum += item
    return sum / df.shape[1]

print(df.apply(avg_3_axis1, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 열 방향

데이터 셋은 seaborn 라이브러리의 타이타닉 데이터 셋을 사용함

In [38]:
import seaborn as sns 
import numpy as np 

titanic = sns.load_dataset('titanic')

In [39]:
# 데이터 셋 정보 확인
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [44]:
# 누락값의 개수를 반환하는 count_missing 함수  

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [47]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [50]:
# 누락값의 비율을 계산하는 함수 
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

In [51]:
cmis_col = titanic.apply(prop_missing)
print(cmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [52]:
# prop_missing 함수를 이용하여 누락값이 아닌 데이터의 비율을 구할 수 있음
def prop_complete(vec):
    return 1 - prop_missing(vec)

In [53]:
cmis_col = titanic.apply(prop_complete)
print(cmis_col)

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive          1.000000
alone          1.000000
dtype: float64


# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 행 방뱡

In [54]:
# axis = 1 로 설정하여 행 방향으로 적용
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

In [55]:
print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [56]:
print(pmis_row.head())

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64


In [57]:
print(pcom_row.head())

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


누락값의 개수를 구하여 titanic 데이터프레임에 추가한 것

In [58]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [60]:
# 누락값이 있는 데이터만 따로 모아서 확인, 누락값이 2개 이상인 데이터만 추출
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex  age  sibsp  parch      fare embarked  class  \
76          0       3    male  NaN      0      0    7.8958        S  Third   
790         0       3    male  NaN      0      0    7.7500        Q  Third   
584         0       3    male  NaN      0      0    8.7125        C  Third   
359         1       3  female  NaN      0      0    7.8792        Q  Third   
825         0       3    male  NaN      0      0    6.9500        Q  Third   
578         0       3  female  NaN      1      0   14.4583        C  Third   
531         0       3    male  NaN      0      0    7.2292        C  Third   
783         0       3    male  NaN      1      2   23.4500        S  Third   
274         1       3  female  NaN      0      0    7.7500        Q  Third   
334         1       1  female  NaN      1      0  133.6500        S  First   

       who  adult_male deck  embark_town alive  alone  num_missing  
76     man        True  NaN  Southampton    no   True            2  
790